*   Victor Muryn
*   Kostyantyn Savorona
*   Yurii Sahaidak

In [ ]:
%pip install -q torch transformers accelerate sentence-transformers faiss-gpu langchain langchain_community peft
%pip install -q httpx==0.27.2 openai tiktoken
%pip install -q git+https://github.com/robinhad/ukrainian-tts.git

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

from google.colab import userdata
import IPython.display as ipd

import time
import pickle

import torch
import numpy as np
from tqdm import tqdm

import tiktoken
from openai import OpenAI

import faiss

from transformers import AutoProcessor, AutoModelForCTC

import scipy.io.wavfile as wavfile
import librosa

from ukrainian_tts.tts import TTS, Voices, Stress

In [ ]:
!gdown 1FLg1UlQuH6E7fzIFeiATsawIby48hQs3
!unzip rag-assistant.zip

In [ ]:
with open('questions.pickle', 'rb') as handle:
    questions = pickle.load(handle)

questions = np.array(questions)

# Speech to Text

In [ ]:
# Load model directly
processor_stt = AutoProcessor.from_pretrained("robinhad/wav2vec2-xls-r-300m-uk")
model_stt = AutoModelForCTC.from_pretrained("robinhad/wav2vec2-xls-r-300m-uk").to("cuda")

In [ ]:
def process_audio(audio_path: str):
  sampling_rate, data = wavfile.read(audio_path)

  if len(data.shape) > 1:
      data = data.mean(axis=1)

  if sampling_rate != 16000:
      data = librosa.resample(data.astype(float), orig_sr=sampling_rate, target_sr=16000)

  audio = torch.tensor(data).to('cuda')
  input_dict = processor_stt(audio, return_tensors="pt", padding=True)
  logits = model_stt(input_dict.input_values.to("cuda")).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]

  return processor_stt.decode(pred_ids)

In [ ]:
%%time
question = process_audio('question.wav')
question

# RAG

### Set up embeddings

In [ ]:
embd_model_e5 = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

In [ ]:
embeddings_e5 = []

for question in tqdm(questions):
  embeddings_e5.append( embd_model_e5.embed_query(question) )

embeddings_e5 = np.float32(embeddings_e5)


100%|██████████| 299/299 [00:09<00:00, 32.75it/s]


In [ ]:
db = faiss.IndexFlatL2(embeddings_e5.shape[1])
db.add(embeddings_e5)

In [ ]:
def get_context_elements(query):
  query_embedding = np.float32([embd_model_e5.embed_query(query)])
  # Perform the search
  D, I = db.search(query_embedding, k=5)  # k is the number of nearest neighbors
  return questions[I][0]

def transform_context(context):
  string = ""

  for i, doc in enumerate(context, start=1):
    string += f"Уривок #{i}: \"{doc}\"\n\n"

  return string.strip()

### Set up LLM

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str = "cl100k_base") -> int:
  encoding = tiktoken.get_encoding(encoding_name)
  num_tokens = len(encoding.encode(string))
  return num_tokens

def price_of_request(string: str, price_per_1m) -> float:
  # https://openai.com/api/pricing/
  price_per_token = price_per_1m / 1_000_000
  tokens = num_tokens_from_string(string)
  return tokens * price_per_token

# $0.50 for input
# $1.50 for output

In [ ]:
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))

In [ ]:
def get_system_prompt(context):
  return f"""Ти – асистент бот у компанії Київстар. Твоє завдання – відповідати на запитання, що стосуються зв'язку та інтернету, виключно на основі наданого контексту.

ПРАВИЛА:
* Відповідай тільки тоді, коли питання має прямий зв’язок із наданим контекстом і контекст містить необхідну інформацію для відповіді.
* Не відповідай, якщо контекст не має відповідної інформації чи не стосується питання.
* Уникай спекуляцій або додавання деталей поза контекстом. Відповідь повинна чітко відповідати на питання на основі контексту.
* Якщо контекст не надано або не містить відповідної інформації, відповідай "Я не маю відповіді на це запитання".

Пам'ятай: відповідай тільки тоді, коли контекст чітко надає інформацію для цього.

Контекст:
{context}
"""

In [ ]:
def get_completion(query):
  context = get_context_elements(query)
  context = transform_context(context)
  system_prompt = get_system_prompt(context)

  completion = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": query
          }
      ]
  )

  # $0.50 per 1M tokens for input
  input_price = completion.usage.prompt_tokens * 0.5 / 1_000_000

  # $1.50 per 1M tokens for output
  output_price = completion.usage.completion_tokens * 1.5 / 1_000_000

  return completion.choices[0].message.content, input_price, output_price

`question` is taken from Speach to text

In [ ]:
question = "як створити унікальний номер з комбінації цифр яка мені подобається"

In [ ]:
%%time
output_text, input_price, output_price = get_completion(question)

CPU times: user 63.1 ms, sys: 6.64 ms, total: 69.8 ms
Wall time: 2.06 s


In [ ]:
input_price + output_price

0.0010255

In [ ]:
print(output_text)

Для створення унікального номера з комбінації цифр, яка вам подобається, ви можете скористатися послугою "Ексклюзивний номер" від Київстар. Ця послуга дозволяє підключити індивідуальний та легко запам'ятовуваний номер. Ви можете підключити цю послугу в інтернет-магазині, найближчому магазині Київстар або партнерських магазинах. Деталі про цю послугу можна знайти в описі на сайті Київстар.


# Text to Speech

In [ ]:
tts = TTS(device="cuda")

In [ ]:
def output_to_voice(output_text: str, output_file: str = "reponse.wav"):
  with open(output_file, mode="wb") as file:
    _, output_text = tts.tts(output_text, Voices.Dmytro.value, Stress.Dictionary.value, file)

  # set speed x1.2
  speed_factor = 1.2
  sampling_rate, data = wavfile.read(output_file)
  new_sampling_rate = int(sampling_rate * speed_factor)
  wavfile.write(output_file, new_sampling_rate, data)

  return output_text

## Final

In [ ]:
def all_together(input_file: str, output_file: str = "reponse.wav"):
  start = time.time()

  question = process_audio(input_file)
  step1 = time.time()

  output_text, input_price, output_price = get_completion(question)
  step2 = time.time()

  accented_text = output_to_voice(output_text, output_file)
  step3 = time.time()

  times = {
      "stt": step1 - start,
      "llm": step2 - step1,
      "tts": step3 - step2,
      "total": step3 - start
  }

  total_price = input_price + output_price

  return question, output_text, (input_price, output_price, total_price), times

In [ ]:
%%time
question, output_text, total_price, times = all_together("question.wav", "reponse.wav")

In [ ]:
print(question)

як я можу створити унікальний номер з цифир які мені подобаються


In [ ]:
print(output_text)

Скористайтеся послугою "Ексклюзивний номер" в інтернет-магазині, найближчому магазині Київстар або магазинах партнерів. Ця послуга дозволить вам вибрати красивий номер, який легко запам'ятовується. Детальну інформацію ви знайдете у описі послуги "Ексклюзивний номер".


In [ ]:
f"${total_price:.6f}"

'$0.000921'

In [ ]:
print(times)

{'stt': 0.04222249984741211, 'llm': 2.2731988430023193, 'tts': 3.525733709335327, 'total': 5.841155052185059}


In [ ]:
ipd.Audio(filename="reponse.wav")

## test times and price

In [ ]:
!unzip -q audios.zip

In [ ]:
!mkdir gpt_responses

In [ ]:
from glob import glob

In [ ]:
wavs = list(glob("./audios/question_*.wav"))
wavs = list(sorted(wavs, key=lambda x: int(x.split("_")[-1].split(".")[0])))

sst_times = []
tts_times = []
llm_times = []
total_times = []

input_prices = []
output_prices = []
total_prices = []


for i, audio_file in tqdm(enumerate(wavs), total=len(wavs)):
  print(i, audio_file)

  question, output_text, (input_price, output_price, total_price), times = all_together(
    audio_file,
    f"./gpt_responses/response_{i}.wav"
  )

  sst_times.append(times['stt'])
  tts_times.append(times['tts'])
  llm_times.append(times['llm'])
  total_times.append(times['total'])

  input_prices.append(input_price)
  output_prices.append(output_price)
  total_prices.append(total_price)

  with open(f"./gpt_responses/response_{i}.txt", 'w') as f:
    f.write(output_text)

  0%|          | 0/51 [00:00<?, ?it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


0 ./audios/question_0.wav


  2%|▏         | 1/51 [00:05<04:15,  5.11s/it]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


RTF = 0.128884
1 ./audios/question_1.wav


  4%|▍         | 2/51 [00:07<02:59,  3.66s/it]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


...

RTF = 0.132219
49 ./audios/question_49.wav


 98%|█████████▊| 50/51 [04:04<00:04,  4.36s/it]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


RTF = 0.126156
50 ./audios/question_50.wav


100%|██████████| 51/51 [04:06<00:00,  4.84s/it]

RTF = 0.127072


In [ ]:
np.mean(sst_times), np.std(sst_times)

(0.04137133617027133, 0.004840655899044445)

In [ ]:
np.mean(llm_times), np.std(llm_times)

(1.7769849347133262, 0.9762185603038583)

In [ ]:
np.mean(tts_times), np.std(tts_times)

(3.016692371929393, 1.9280179571154974)

In [ ]:
np.mean(total_times), np.std(total_times)

(4.835048642812991, 2.8322782385275254)

In [ ]:
np.mean(input_prices), np.std(input_prices)

(0.0008988627450980392, 0.0002829277791591264)

In [ ]:
np.mean(output_prices), np.std(output_prices)

(0.00027282352941176475, 0.00017719123444463568)

In [ ]:
np.mean(total_prices), np.std(total_prices)

(0.001171686274509804, 0.00040619260331532564)

In [ ]:
sum(total_prices)

0.059756000000000004

In [ ]:
!zip -r gpt_responses.zip gpt_responses